In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [2]:
import scvelo as scv
dataset = 'Dentategyrus'
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')

In [ ]:
vv.preprocess(adata, n_gene=2000, min_shared_counts=20)

In [3]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'data/velovae/discrete/{dataset}'
gene_plot = ['Tmsb10', 'Fam155a', 'Hn1', 'Rpl6']

# VeloVAE

In [4]:
figure_path = f'{figure_path_base}/DVAE'
model_path = f'{model_path_base}/DVAE'

torch.manual_seed(2022)
np.random.seed(2022)
vae = vv.DVAE(adata, 
              tmax=20, 
              dim_z=5, 
              device='cuda:0',
              init_method='random')

vae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

vae.save_model(model_path, 'encoder', 'decoder')
vae.save_anndata(adata, 'dvae', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Random Initialization.
Gaussian Prior.
Library scale (U): Max=6.46, Min=0.03, Mean=0.88
Library scale (S): Max=6.53, Min=0.29, Mean=1.02
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********       Stage 1: Early Stop Triggered at epoch 546.       *********
*********                      Stage  2                       *********


  0%|          | 0/2930 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.002
Average Set Size: 69
*********       Stage 2: Early Stop Triggered at epoch 569.       *********
*********              Finished. Total Time =   0 h :  2 m : 24 s             *********
Final: Train ELBO = -1729.301,           Test ELBO = -1713.168
       Training MSE = 1.798, Test MSE = 2.064


# Full VB

In [5]:
figure_path = f'{figure_path_base}/DFullVB'
model_path = f'{model_path_base}/DFullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.DVAEFullVB(adata, 
                        tmax=20, 
                        dim_z=5, 
                        device='cuda:0',
                        init_method='random')

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'dfullvb', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Random Initialization.
Gaussian Prior.
Library scale (U): Max=6.46, Min=0.03, Mean=0.88
Library scale (S): Max=6.53, Min=0.29, Mean=1.02
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********       Stage 1: Early Stop Triggered at epoch 1392.       *********
*********                      Stage  2                       *********


  0%|          | 0/2930 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.001
Average Set Size: 112
*********       Stage 2: Early Stop Triggered at epoch 1432.       *********
*********              Finished. Total Time =   0 h :  6 m : 47 s             *********
Final: Train ELBO = -1749.321,           Test ELBO = -1749.294
       Training MSE = 1.858, Test MSE = 2.188


# Evaluation

In [ ]:
cluster_edges = [('OPC', 'OL')]
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE','Discrete FullVB'],
                 ['dvae','dfullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)